# Introduction to Building LLM Agents with Tools and Tracing

This script walks through the process of building a simple LLM-powered agent that can use tools (functions) to answer questions. We'll cover:
1. Making basic LLM calls.
2. Introducing Weave for tracing and observability.
3. Defining tools for the LLM (manually and automatically).
4. Implementing a basic agentic loop.
5. Structuring the agent using Python classes.
6. Agent evaluation

**Prerequisites:**
Make sure you have the necessary libraries installed:
```bash
!pip install weave requests boto3
```


In [5]:
# Global Configuration & Setup
import inspect
import json
import os
import requests
import weave # Must import weave before litellm for auto-patching
from enum import Enum
from pydantic import BaseModel, Field
from rich.pretty import pprint
from typing import Any, Callable, Dict, List, get_type_hints

In [6]:
# you need to request access to the model on the Bedrock UI
CLAUDE_MODEL = "us.anthropic.claude-sonnet-4-20250514-v1:0"

Define a model to use, as we are going to use tool calling you need a capable model like `claude-4`

Let's log to [W&B Weave](https://weave-docs.wandb.ai/). Weights & Biases (W&B) Weave is a framework for tracking, experimenting with, evaluating, deploying, and improving LLM-based applications. Designed for flexibility and scalability, Weave supports every stage of your LLM application development workflow:

- Tracing & Monitoring: Track LLM calls and application logic to debug and analyze production systems.
- Systematic Iteration: Refine and iterate on prompts, datasets, and models.
- Experimentation: Experiment with different models and prompts in the LLM Playground.
- Evaluation: Use custom or pre-built scorers alongside our comparison tools to systematically assess and enhance application performance.
- Guardrails: Protect your application with pre- and post-safeguards for content moderation, prompt safety, and more.

In [7]:

# Initialize a Weave project. Traces will be sent here.
# You can view them in the Weave UI (usually runs locally).
weave.init('reliable-agents')

## 1. Basic LLM Call with Bedrock Converse API 

Let's start with a simple call to the LLM using/

![](br_images/01_traces.png)

In [8]:
import boto3
from weave.integrations.bedrock.bedrock_sdk import patch_client

# Create and patch the Bedrock client
client = boto3.client("bedrock-runtime")
patch_client(client)

messages = [{"role": "user", "content": [{"text": "Hello, LLM! How does an AI agent work?"}]}]

# call the model using the converse API
response = client.converse(
    modelId=CLAUDE_MODEL,
    system=[{"text": "You are a helpful AI assistant."}],
    messages=messages,
    inferenceConfig={"maxTokens": 400},
)
pprint(response)

{
│   'ResponseMetadata': {
│   │   'RequestId': 'cd346d86-f30f-4957-a98d-b6910a438030',
│   │   'HTTPStatusCode': 200,
│   │   'HTTPHeaders': {
│   │   │   'date': 'Mon, 30 Jun 2025 15:49:51 GMT',
│   │   │   'content-type': 'application/json',
│   │   │   'content-length': '1810',
│   │   │   'connection': 'keep-alive',
│   │   │   'x-amzn-requestid': 'cd346d86-f30f-4957-a98d-b6910a438030'
│   │   },
│   │   'RetryAttempts': 0
│   },
│   'output': {
│   │   'message': {
│   │   │   'role': 'assistant',
│   │   │   'content': [
│   │   │   │   {
│   │   │   │   │   'text': "Hello! Great question about AI agents. An AI agent is essentially a system that can perceive its environment, make decisions, and take actions to achieve specific goals. Here's how they typically work:\n\n## Core Components:\n\n**1. Perception**\n- Sensors or input mechanisms to gather information from the environment\n- Could be text input, visual data, audio, or other data types\n\n**2. Decision-Making**\n- Processing system (like neural networks, rule-based systems, or hybrid approaches)\n- Evaluates current state and determines best course of action\n- Often uses models trained on large datasets\n\n**3. Action**\n- Output mechanisms to interact with the environment\n- Could be generating text, controlling robotics, making API calls, etc.\n\n## Basic Process Flow:\n1. **Observe** → Gather current environmental information\n2. **Think** → Process information using internal models/algorithms\n3. **Act** → Execute chosen action\n4. **Learn** → Update knowledge based on results (in learning agents)\n5. **Repeat** → Continue the cycle\n\n## Types of AI Agents:\n- **Reactive**: Respond directly to current inputs\n- **Deliberative**: Plan ahead using internal models\n- **Learning**: Improve performance over time\n- **Autonomous**: Operate independently with minimal human intervention\n\nThe specific implementation varies greatly depending on the application - from chatbots like me to autonomous vehicles to game-playing systems!\n\nWhat particular aspect of AI agents interests you most?"
│   │   │   │   }
│   │   │   ]
│   │   }
│   },
│   'stopReason': 'end_turn',
│   'usage': {
│   │   'inputTokens': 27,
│   │   'outputTokens': 327,
│   │   'totalTokens': 354,
│   │   'cacheReadInputTokens': 0,
│   │   'cacheWriteInputTokens': 0
│   },
│   'metrics': {'latencyMs': 8312}
}

To get your LLM calls traces, you need to call `patch_client(client)` on the `boto3` Bedrock Runtime client. You can follow the link to your Weave dashboard and see the trace, including the input messages, output response, latency, model used, etc. This is invaluable for debugging and monitoring.

In [9]:
# most of the time you would want to define your own operations to trace, for instance to call the model.
# You just need to add the @weave.op decorator to the function and it will be traced.

@weave.op
def call_model(model_name: str, messages: List[Dict[str, Any]], system: List[Dict[str, Any]]=[{"text": "You are a helpful AI assistant."}], **kwargs) -> str:
    "Call a model with the given messages and kwargs."

    # call the model using the converse API
    response = client.converse(
        modelId=CLAUDE_MODEL,
        system=system,
        messages=messages,
        inferenceConfig={"maxTokens": 400},
        **kwargs
    )
    return response

response = call_model(model_name=CLAUDE_MODEL, messages=messages)
pprint(response)

{
│   'ResponseMetadata': {
│   │   'RequestId': '51159541-2379-4e30-b505-6b7ba6d217f8',
│   │   'HTTPStatusCode': 200,
│   │   'HTTPHeaders': {
│   │   │   'date': 'Mon, 30 Jun 2025 15:52:04 GMT',
│   │   │   'content-type': 'application/json',
│   │   │   'content-length': '1731',
│   │   │   'connection': 'keep-alive',
│   │   │   'x-amzn-requestid': '51159541-2379-4e30-b505-6b7ba6d217f8'
│   │   },
│   │   'RetryAttempts': 0
│   },
│   'output': {
│   │   'message': {
│   │   │   'role': 'assistant',
│   │   │   'content': [
│   │   │   │   {
│   │   │   │   │   'text': "Hello! Great question about AI agents. Here's how they typically work:\n\n## Core Components\n\n**1. Perception**\n- Sensors or input mechanisms to gather information from their environment\n- Could be text input, visual data, audio, or other data sources\n\n**2. Decision-Making**\n- Processing unit (like a neural network) that analyzes the perceived information\n- Uses learned patterns, rules, or models to determine appropriate actions\n\n**3. Action**\n- Output mechanisms to interact with or modify their environment\n- Could be generating text, controlling hardware, making API calls, etc.\n\n## Basic Operation Cycle\n\n1. **Observe** the current state of the environment\n2. **Process** that information using internal models/knowledge\n3. **Decide** on the best action based on goals and constraints\n4. **Act** by executing the chosen action\n5. **Learn** from the results (in adaptive systems)\n6. **Repeat** the cycle\n\n## Key Characteristics\n\n- **Goal-oriented**: Work toward specific objectives\n- **Autonomous**: Can operate independently once deployed\n- **Reactive**: Respond to changes in their environment\n- **Adaptive**: Many can learn and improve over time\n\nThe complexity can range from simple rule-based systems to sophisticated AI models like large language models that can handle complex reasoning and natural language interaction.\n\nWould you like me to elaborate on any particular aspect of AI agents?"
│   │   │   │   }
│   │   │   ]
│   │   }
│   },
│   'stopReason': 'end_turn',
│   'usage': {
│   │   'inputTokens': 27,
│   │   'outputTokens': 308,
│   │   'totalTokens': 335,
│   │   'cacheReadInputTokens': 0,
│   │   'cacheWriteInputTokens': 0
│   },
│   'metrics': {'latencyMs': 7424}
}

🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197c18f-d774-76c1-bd8e-e2ac72a79908
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197c1a2-832a-7731-a471-327382b4cfca


![](br_images/02_nested_tracing.png)


## 2. Introducing Tool Calling

Agents become much more powerful when they can use **tools** – external functions or APIs – to get information or perform actions beyond the LLM's internal knowledge. To allow an LLM to use a tool, we need to provide it with a description (schema) of the tool, including its name, purpose, and expected arguments.


![](br_images/03_tools.png)

First, let's define a simple Python function we want the LLM to be able to call. We add `@weave.op` to trace when this function actually gets executed.


In [10]:
@weave.op 
def add_numbers(a: int, b: int) -> int:
    """Adds two numbers.
    Args:
        a: The first number.
        b: The second number.
    """
    return a + b

In [11]:
add_numbers(1, 2)

3

🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197c197-45d8-7462-8a86-0a1c1bfcd520
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197c19d-2696-7ec3-af53-382bcd4ba8f0
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197c1a2-81d8-7ce1-9451-f4ceae468649


In [12]:
# this doesn't work...
call_model(model_name=CLAUDE_MODEL, messages=messages, tools=[add_numbers])

ParamValidationError: Parameter validation failed:
Unknown parameter in input: "tools", must be one of: modelId, messages, system, inferenceConfig, toolConfig, guardrailConfig, additionalModelRequestFields, promptVariables, additionalModelResponseFieldPaths, requestMetadata, performanceConfig

> We need to manually create the JSON schema describing this tool in a format that models *Claude* understand.

In [13]:
# Manually define the tool schema
tool_add_numbers_schema = {
  "toolSpec": {
    "name": "add_numbers",
    "description": "Adds two numbers.",
    "inputSchema": {
      "json": {
        "type": "object",
        "properties": {
          "a": {
            "type": "integer",
            "description": "The first number."
          },
          "b": {
            "type": "integer",
            "description": "The second number."
          }
        },
        "required": ["a", "b"]
      }
    }
  }
}


tool_config = { "tools": [tool_add_numbers_schema] }

In [14]:
tool_config

{'tools': [{'toolSpec': {'name': 'add_numbers',
    'description': 'Adds two numbers.',
    'inputSchema': {'json': {'type': 'object',
      'properties': {'a': {'type': 'integer',
        'description': 'The first number.'},
       'b': {'type': 'integer', 'description': 'The second number.'}},
      'required': ['a', 'b']}}}}]}

Now, we make an LLM call, passing the `tools` parameter with our schema. We ask a question that should trigger the tool.

In [15]:
system = [{"text": "You are a helpful assistant that can use tools to answer questions."}]
messages = [{"role": "user", "content": [{"text": "My lucky numbers are 77 and 11. What is their sum?"}]}]
response = call_model(model_name=CLAUDE_MODEL, messages=messages, system=system, toolConfig=tool_config)
pprint(response)

{
│   'ResponseMetadata': {
│   │   'RequestId': '39e6195c-a709-484c-adec-ffda306d9081',
│   │   'HTTPStatusCode': 200,
│   │   'HTTPHeaders': {
│   │   │   'date': 'Mon, 30 Jun 2025 15:56:15 GMT',
│   │   │   'content-type': 'application/json',
│   │   │   'content-length': '457',
│   │   │   'connection': 'keep-alive',
│   │   │   'x-amzn-requestid': '39e6195c-a709-484c-adec-ffda306d9081'
│   │   },
│   │   'RetryAttempts': 0
│   },
│   'output': {
│   │   'message': {
│   │   │   'role': 'assistant',
│   │   │   'content': [
│   │   │   │   {'text': "I'll help you find the sum of your lucky numbers 77 and 11."},
│   │   │   │   {
│   │   │   │   │   'toolUse': {
│   │   │   │   │   │   'toolUseId': 'tooluse_2824EyadTHWaUX1U9n3O0Q',
│   │   │   │   │   │   'name': 'add_numbers',
│   │   │   │   │   │   'input': {'a': 77, 'b': 11}
│   │   │   │   │   }
│   │   │   │   }
│   │   │   ]
│   │   }
│   },
│   'stopReason': 'tool_use',
│   'usage': {
│   │   'inputTokens': 431,
│   │   'outputTokens': 89,
│   │   'totalTokens': 520,
│   │   'cacheReadInputTokens': 0,
│   │   'cacheWriteInputTokens': 0
│   },
│   'metrics': {'latencyMs': 2356}
}

## Manual Tool Call
The LLM's response might contain a request to call our tool. If it requests a tool call, we need to:

1. Parse the arguments it provides.
2. Execute our actual Python function (`add_numbers`) with those arguments.
3. (In a real agent loop) Send the result back to the LLM in a new message with `role="tool"`.

Let's manually call the tools in the response.

In [16]:
if response["stopReason"] == "tool_use":
    print("LLM requested a tool call:")
    for message in response["output"]["message"]["content"]:
        if message.get("text"):
            print(message["text"])
        if message.get("toolUse", False):
            tool_name = message["toolUse"]["name"]
            tool_args = message["toolUse"]["input"]
            tool_use_id = message["toolUse"]["toolUseId"]
            print(f"  - Tool: {tool_name}, Args: {tool_args}")
            if tool_name == "add_numbers":
                tool_result_content = add_numbers(**tool_args)

print(tool_result_content)

LLM requested a tool call:
I'll help you find the sum of your lucky numbers 77 and 11.
  - Tool: add_numbers, Args: {'a': 77, 'b': 11}
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197c18f-008e-7e62-b51a-c70fd2dd4674
88
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197c194-5c06-7b73-b9ab-72a23485d29a
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197c1a2-8072-7271-a205-3dd59cd62ef0


We need to add the tool call result to the messages (there is actually 2 messages to add)
- the response from the assistant that decided to call the tool
- the tool output

In [17]:
messages.append(response["output"]["message"])

In [18]:
messages

[{'role': 'user',
  'content': [{'text': 'My lucky numbers are 77 and 11. What is their sum?'}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you find the sum of your lucky numbers 77 and 11."},
   {'toolUse': {'toolUseId': 'tooluse_2824EyadTHWaUX1U9n3O0Q',
     'name': 'add_numbers',
     'input': {'a': 77, 'b': 11}}}]}]

In [19]:
tool_result = {
        "toolUseId": tool_use_id,
        "status": "success",
        "content": [{"json": {"sum": tool_result_content}}]
}

In [20]:
messages.append({"role": "user", "content": [{"toolResult": tool_result}]})

In [21]:
messages

[{'role': 'user',
  'content': [{'text': 'My lucky numbers are 77 and 11. What is their sum?'}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you find the sum of your lucky numbers 77 and 11."},
   {'toolUse': {'toolUseId': 'tooluse_2824EyadTHWaUX1U9n3O0Q',
     'name': 'add_numbers',
     'input': {'a': 77, 'b': 11}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_2824EyadTHWaUX1U9n3O0Q',
     'status': 'success',
     'content': [{'json': {'sum': 88}}]}}]}]

Now call the model again with the new messages and it will use the tool call result to answer the question

In [22]:
final_response = call_model(model_name=CLAUDE_MODEL, messages=messages, system=system, toolConfig=tool_config)
pprint(final_response)

{
│   'ResponseMetadata': {
│   │   'RequestId': 'c779428c-05e4-41a9-93b5-8308aa7774cc',
│   │   'HTTPStatusCode': 200,
│   │   'HTTPHeaders': {
│   │   │   'date': 'Mon, 30 Jun 2025 15:58:40 GMT',
│   │   │   'content-type': 'application/json',
│   │   │   'content-length': '340',
│   │   │   'connection': 'keep-alive',
│   │   │   'x-amzn-requestid': 'c779428c-05e4-41a9-93b5-8308aa7774cc'
│   │   },
│   │   'RetryAttempts': 0
│   },
│   'output': {
│   │   'message': {'role': 'assistant', 'content': [{'text': 'The sum of your lucky numbers 77 and 11 is 88.'}]}
│   },
│   'stopReason': 'end_turn',
│   'usage': {
│   │   'inputTokens': 536,
│   │   'outputTokens': 20,
│   │   'totalTokens': 556,
│   │   'cacheReadInputTokens': 0,
│   │   'cacheWriteInputTokens': 0
│   },
│   'metrics': {'latencyMs': 1401}
}

🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197c197-bd39-73e0-8c58-7fa170e8d222
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197c199-62c5-7df3-8053-70d0ed79adb9
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197c1a3-20d6-7921-906e-da56e37df2e3
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197c1a5-6c2a-76f1-b635-43c5ba171051


## 3. Simplifying Tool Definition with a Processor Function

Manually writing JSON schemas is tedious and error-prone. We can automate this by inspecting our Python function's signature, type hints, and docstring.

First, let's define a helper function (`generate_tool_schema`) that takes a Python function and generates the schema.


In [29]:
def generate_tool_schema(func: Callable) -> dict:
    """Given a Python function, generate a Bedrock-compatible JSON schema.
    Handles basic types and Enums. Assumes docstrings are formatted for arg descriptions.
    """
    signature = inspect.signature(func)
    parameters = signature.parameters
    type_hints = get_type_hints(func)

    # Extract main description (everything before Args: section)
    docstring = inspect.getdoc(func)
    main_description = ""
    if docstring:
        lines = docstring.split('\n')
        for line in lines:
            if line.strip().lower().startswith(("args:", "arguments:", "parameters:")):
                break
            if main_description:
                main_description += " "
            main_description += line.strip()

    schema = {
        "toolSpec": {
            "name": func.__name__,
            "description": main_description,
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                }
            }
        }
    }

    # Parse parameter descriptions from docstring
    param_descriptions = {}
    if docstring:
        args_section = False
        for line in docstring.split('\n'):
            line_stripped = line.strip()
            if line_stripped.lower().startswith(("args:", "arguments:", "parameters:")):
                args_section = True
                continue
            if args_section:
                if ":" in line_stripped and line_stripped[0] != ' ':
                    param_name, desc = line_stripped.split(":", 1)
                    param_descriptions[param_name.strip()] = desc.strip()
                elif line_stripped and not line_stripped.startswith(" ") and ":" not in line_stripped:
                    # End of args section
                    args_section = False

    for name, param in parameters.items():
        is_required = param.default == inspect.Parameter.empty
        param_type = type_hints.get(name, Any)
        json_type = "string"
        param_schema = {}

        # Basic type mapping
        if param_type == str: json_type = "string"
        elif param_type == int: json_type = "integer"
        elif param_type == float: json_type = "number"
        elif param_type == bool: json_type = "boolean"
        elif hasattr(param_type, '__origin__') and param_type.__origin__ is list: # Handle List[type]
             item_type = param_type.__args__[0] if param_type.__args__ else Any
             if item_type == str: param_schema = {"type": "array", "items": {"type": "string"}}
             elif item_type == int: param_schema = {"type": "array", "items": {"type": "integer"}}
             # Add more list item types if needed
             else: param_schema = {"type": "array", "items": {"type": "string"}} # Default list item type
        elif hasattr(param_type, "__members__") and issubclass(param_type, Enum): # Handle Enum
             json_type = "string"
             param_schema["enum"] = [e.value for e in param_type]

        if not param_schema: # If not set by List or Enum
            param_schema["type"] = json_type

        param_schema["description"] = param_descriptions.get(name, "")

        if param.default != inspect.Parameter.empty and param.default is not None:
             param_schema["default"] = param.default # Note: Bedrock schema doesn't officially use default, but useful metadata

        schema["toolSpec"]["inputSchema"]["json"]["properties"][name] = param_schema
        if is_required:
            schema["toolSpec"]["inputSchema"]["json"]["required"].append(name)
    return schema

Now we can use this function to automatically generate the schema for our tool.

In [30]:
tool_schema = generate_tool_schema(add_numbers)
pprint(tool_schema)

{
│   'toolSpec': {
│   │   'name': 'add_numbers',
│   │   'description': 'Adds two numbers.',
│   │   'inputSchema': {
│   │   │   'json': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {
│   │   │   │   │   'a': {'type': 'integer', 'description': 'The first number.'},
│   │   │   │   │   'b': {'type': 'integer', 'description': 'The second number.'}
│   │   │   │   },
│   │   │   │   'required': ['a', 'b']
│   │   │   }
│   │   }
│   }
}

Now, we define a `function_tool` "processor". This isn't a decorator in the `@` syntax sense here, but a function that we call *after* defining our tool function. It uses `generate_tool_schema` to attach the schema to the function object itself.


In [31]:
def function_tool(func: Callable) -> Callable:
    """Attaches a tool schema to the function and marks it as a tool.
    Call this *after* defining your function: my_func = function_tool(my_func)
    """
    try:
        func.tool_schema = generate_tool_schema(func)
        func.is_tool = True # Mark it as a tool
    except Exception as e:
        print(f"Error processing tool {func.__name__}: {e}")
        # Optionally raise or mark as failed
        func.tool_schema = None
        func.is_tool = False
    return func

We can use this function to automatically generate the schema for our tool, as a decorator or after the function is defined.

In [32]:
add_numbers = function_tool(add_numbers)
pprint(add_numbers.tool_schema)


{
│   'toolSpec': {
│   │   'name': 'add_numbers',
│   │   'description': 'Adds two numbers.',
│   │   'inputSchema': {
│   │   │   'json': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {
│   │   │   │   │   'a': {'type': 'integer', 'description': 'The first number.'},
│   │   │   │   │   'b': {'type': 'integer', 'description': 'The second number.'}
│   │   │   │   },
│   │   │   │   'required': ['a', 'b']
│   │   │   }
│   │   }
│   }
}

In [33]:
add_numbers.tool_schema

{'toolSpec': {'name': 'add_numbers',
  'description': 'Adds two numbers.',
  'inputSchema': {'json': {'type': 'object',
    'properties': {'a': {'type': 'integer',
      'description': 'The first number.'},
     'b': {'type': 'integer', 'description': 'The second number.'}},
    'required': ['a', 'b']}}}}

and call the tool =)

In [34]:
add_numbers(1, 2)

3

🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197c1a2-8108-78e1-a534-15821e24a2d9
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197c1a4-a4e9-74b1-b19b-277262c7a3e6


### 3.1 Real Example using an API based tool

Let's create a real example hitting an API: The pokemon API

In [35]:
@weave.op 
@function_tool # <- we can use the decorator to automatically generate the tool schema
def get_pokemon_info(pokemon_name: str) -> str:
    """Fetches minimal information (name, ID, weight) for a specific Pokemon. 
    - Weight is in hectograms.
    - Height is in decimeters.
    - Abilities are the abilities of the Pokemon.
    - Stats are the stats of the Pokemon.
    
    This is a tool that can be used to get information about a specific Pokemon.

    Args:
        pokemon_name: The name or Pokedex ID of the Pokemon.
    """
    base_url = "https://pokeapi.co/api/v2/pokemon/"
    try:
        response = requests.get(f"{base_url}{pokemon_name.lower().strip()}")
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        info = {
            "name": data.get('name', 'Unknown').capitalize(),
            "id": data.get('id', -1),
            "weight": data.get('weight', -1), # Weight in hectograms
            "height": data.get('height', -1), # Height in decimeters
            "abilities": [a["ability"]["name"] for a in data.get('abilities', [])],
            "stats": {s["stat"]["name"]:s["base_stat"] for s in data.get('stats', {})},
        }
        return info
    except Exception as e:
        return {"error": f"Error fetching pokemon {pokemon_name}: {str(e)}"}

In [36]:
get_pokemon_info.tool_schema

{'toolSpec': {'name': 'get_pokemon_info',
  'description': 'Fetches minimal information (name, ID, weight) for a specific Pokemon. - Weight is in hectograms. - Height is in decimeters. - Abilities are the abilities of the Pokemon. - Stats are the stats of the Pokemon.  This is a tool that can be used to get information about a specific Pokemon. ',
  'inputSchema': {'json': {'type': 'object',
    'properties': {'pokemon_name': {'type': 'string',
      'description': 'The name or Pokedex ID of the Pokemon.'}},
    'required': ['pokemon_name']}}}}

In [37]:
get_pokemon_info("pikachu")

{'name': 'Pikachu',
 'id': 25,
 'weight': 60,
 'height': 4,
 'abilities': ['static', 'lightning-rod'],
 'stats': {'hp': 35,
  'attack': 55,
  'defense': 40,
  'special-attack': 50,
  'special-defense': 50,
  'speed': 90}}

In [38]:
system = [{"text": "You are a helpful assistant that can use tools to answer questions."}]
messages = [{"role": "user", "content": [{"text":"What is the weight of Pikachu?"}]}]
tool_config = { "tools": [get_pokemon_info.tool_schema] }

response = call_model(model_name=CLAUDE_MODEL, messages=messages, system=system, toolConfig=tool_config)
pprint(response)

{
│   'ResponseMetadata': {
│   │   'RequestId': 'c0a1b7a9-5591-42bd-8c85-9a97e1a023d9',
│   │   'HTTPStatusCode': 200,
│   │   'HTTPHeaders': {
│   │   │   'date': 'Mon, 30 Jun 2025 16:06:48 GMT',
│   │   │   'content-type': 'application/json',
│   │   │   'content-length': '452',
│   │   │   'connection': 'keep-alive',
│   │   │   'x-amzn-requestid': 'c0a1b7a9-5591-42bd-8c85-9a97e1a023d9'
│   │   },
│   │   'RetryAttempts': 0
│   },
│   'output': {
│   │   'message': {
│   │   │   'role': 'assistant',
│   │   │   'content': [
│   │   │   │   {'text': "I'll look up Pikachu's weight for you."},
│   │   │   │   {
│   │   │   │   │   'toolUse': {
│   │   │   │   │   │   'toolUseId': 'tooluse_H28VYWgETTGOXTipAcwdfQ',
│   │   │   │   │   │   'name': 'get_pokemon_info',
│   │   │   │   │   │   'input': {'pokemon_name': 'Pikachu'}
│   │   │   │   │   }
│   │   │   │   }
│   │   │   ]
│   │   }
│   },
│   'stopReason': 'tool_use',
│   'usage': {
│   │   'inputTokens': 483,
│   │   'outputTokens': 74,
│   │   'totalTokens': 557,
│   │   'cacheReadInputTokens': 0,
│   │   'cacheWriteInputTokens': 0
│   },
│   'metrics': {'latencyMs': 2141}
}

Let's create some helper functions to perform the tool calls

In [39]:
def get_tool(tools: list[Callable], name: str) -> Callable:
    for t in tools:
        if t.__name__ == name:
            return t
    raise KeyError(f"No tool with name {name} found")

def perform_tool_calls(tools: list[Callable], response: dict) -> list[dict]:
    """Perform the tool calls from a Bedrock response and return the messages with the tool call results"""
    tool_content = []
    if response["stopReason"] == "tool_use":
        print("LLM requested tool calls:")
        for message in response["output"]["message"]["content"]:
            if message.get("text"):
                print(message["text"])
            if message.get("toolUse", False):
                tool_name = message["toolUse"]["name"]
                tool_args = message["toolUse"]["input"]
                tool_use_id = message["toolUse"]["toolUseId"]
                print(f"  - Tool: {tool_name}, Args: {tool_args}")
                
                try:
                    # Get the tool function and call it
                    tool = get_tool(tools, tool_name)
                    tool_response = tool(**tool_args)
                    print(f"  - Response: {tool_response}")
                    
                    # Format the tool result for Bedrock
                    tool_result = {
                        "toolUseId": tool_use_id,
                        "status": "success",
                        "content": [{"json": {"result": tool_response}}]
                    }
                    
                    # Add the tool result message
                    tool_content.append({"toolResult": tool_result})
                    
                except Exception as e:
                    print(f"  - Error: {e}")
                    # Format error response
                    tool_result = {
                        "toolUseId": tool_use_id,
                        "status": "error",
                        "content": [{"text": f"Error executing tool: {str(e)}"}]
                    }
                    tool_content.append({"toolResult": tool_result})


    return {"role": "user", "content": tool_content}

In [40]:

# add the tool call result to the messages
messages.append(response["output"]["message"])

# add the tool call result to the messages
messages.append(perform_tool_calls(tools=[get_pokemon_info], response=response))

LLM requested tool calls:
I'll look up Pikachu's weight for you.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'Pikachu'}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60, 'height': 4, 'abilities': ['static', 'lightning-rod'], 'stats': {'hp': 35, 'attack': 55, 'defense': 40, 'special-attack': 50, 'special-defense': 50, 'speed': 90}}


In [41]:
messages

[{'role': 'user', 'content': [{'text': 'What is the weight of Pikachu?'}]},
 {'role': 'assistant',
  'content': [{'text': "I'll look up Pikachu's weight for you."},
   {'toolUse': {'toolUseId': 'tooluse_H28VYWgETTGOXTipAcwdfQ',
     'name': 'get_pokemon_info',
     'input': {'pokemon_name': 'Pikachu'}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_H28VYWgETTGOXTipAcwdfQ',
     'status': 'success',
     'content': [{'json': {'result': {'name': 'Pikachu',
         'id': 25,
         'weight': 60,
         'height': 4,
         'abilities': ['static', 'lightning-rod'],
         'stats': {'hp': 35,
          'attack': 55,
          'defense': 40,
          'special-attack': 50,
          'special-defense': 50,
          'speed': 90}}}}]}}]}]

In [42]:
final_response = call_model(model_name=CLAUDE_MODEL, messages=messages, toolConfig=tool_config)
pprint(final_response)


{
│   'ResponseMetadata': {
│   │   'RequestId': '889a244b-0d68-4685-9d24-ade66750fdbf',
│   │   'HTTPStatusCode': 200,
│   │   'HTTPHeaders': {
│   │   │   'date': 'Mon, 30 Jun 2025 16:07:27 GMT',
│   │   │   'content-type': 'application/json',
│   │   │   'content-length': '379',
│   │   │   'connection': 'keep-alive',
│   │   │   'x-amzn-requestid': '889a244b-0d68-4685-9d24-ade66750fdbf'
│   │   },
│   │   'RetryAttempts': 0
│   },
│   'output': {
│   │   'message': {
│   │   │   'role': 'assistant',
│   │   │   'content': [
│   │   │   │   {'text': 'Pikachu weighs 60 hectograms, which is equivalent to 6 kilograms (about 13.2 pounds).'}
│   │   │   ]
│   │   }
│   },
│   'stopReason': 'end_turn',
│   'usage': {
│   │   'inputTokens': 628,
│   │   'outputTokens': 35,
│   │   'totalTokens': 663,
│   │   'cacheReadInputTokens': 0,
│   │   'cacheWriteInputTokens': 0
│   },
│   'metrics': {'latencyMs': 1682}
}

Let's wrap this in a function and add a few more tools.

In [43]:
@weave.op
def pokedex(pokemon_question: str) -> str:
    system = [{"text": "You are a helpful assistant that can use tools to answer questions."}]
    messages = [{"role": "user", "content": [{"text": pokemon_question}]}]
    
    # Create tool config for Bedrock
    tool_config = {"tools": [get_pokemon_info.tool_schema]}

    # call model with tools
    response = call_model(
        model_name=CLAUDE_MODEL, 
        messages=messages,
        system=system,
        toolConfig=tool_config
    )

    # add the assistant's response to the messages
    messages.append(response["output"]["message"])
    # pprint(messages)

    # perform the tool calls if any
    tool_result_messages = perform_tool_calls(tools=[get_pokemon_info], response=response)
    messages.append(tool_result_messages)
    # pprint(messages)

    # get final response
    final_response = call_model(
        model_name=CLAUDE_MODEL, 
        messages=messages,
        system=system,
        toolConfig=tool_config
    )
    
    return final_response["output"]["message"]["content"][-1]["text"]

In [44]:
response = pokedex("What is the combined weight of Ash's first 3 pokemons?")
print(response)

LLM requested tool calls:
I'll help you find the combined weight of Ash's first 3 Pokémon. Ash's first three Pokémon were Pikachu, Caterpie, and Pidgeotto. Let me get their weight information.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'pikachu'}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60, 'height': 4, 'abilities': ['static', 'lightning-rod'], 'stats': {'hp': 35, 'attack': 55, 'defense': 40, 'special-attack': 50, 'special-defense': 50, 'speed': 90}}
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'caterpie'}
  - Response: {'name': 'Caterpie', 'id': 10, 'weight': 29, 'height': 3, 'abilities': ['shield-dust', 'run-away'], 'stats': {'hp': 45, 'attack': 30, 'defense': 35, 'special-attack': 20, 'special-defense': 20, 'speed': 45}}
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'pidgeotto'}
  - Response: {'name': 'Pidgeotto', 'id': 17, 'weight': 300, 'height': 11, 'abilities': ['keen-eye', 'tangled-feet', 'big-pecks'], 'stats': {'hp': 63, 'attack': 60, 'defense':

![](br_images/04_pokedex.png)

This is "Almost" an agent, but it's missing the loop. Let's add that next.

## 4. Implementing a Basic Agentic Loop

Let's implement a basic agentic loop. We'll use the `pokedex` function we just created. The implementation we have above has some limitations:
- Its a single turn, so if it fails to answer my question in one pass it is over.

![](images/05_agent.png)

From the really good [Anthropic Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents) article and encourage people to read it.

A simple for loop

In [45]:
@weave.op
def pokedex_loop(pokemon_question: str, max_turns: int = 4, tools = [get_pokemon_info, add_numbers]) -> str:
    system = [{"text": "You are a helpful assistant that can use tools to answer questions."}]
    messages = [{"role": "user", "content": [{"text": pokemon_question}]}]
    
    # Create tool config for Bedrock
    tool_config = {"tools": [t.tool_schema for t in tools]}
    
    for turn in range(max_turns):
        print(f"--- Agent Loop Turn {turn + 1}/{max_turns} ---")

        # call model with tools
        response = call_model(
            model_name=CLAUDE_MODEL, 
            messages=messages,
            system=system,
            toolConfig=tool_config
        )

        # add the assistant's response to the messages
        messages.append(response["output"]["message"])

        # if the LLM requested tool calls, perform them
        if response["stopReason"] == "tool_use":
            print("LLM requested tool calls:")
            # perform the tool calls
            tool_result_messages = perform_tool_calls(tools=tools, response=response)
            messages.append(tool_result_messages)
        # LLM gave content response
        elif response["stopReason"] == "end_turn":
            # Extract text content from response
            content = ""
            for content_item in response["output"]["message"]["content"]:
                if content_item.get("text"):
                    content += content_item["text"]
            print(f"LLM content response: {content}")
            return content
        else:
            print(f"LLM response had unexpected stop reason: {response['stopReason']}. Stopping loop.")
            break
    
    # If we reach max_turns without a final response, return the last assistant message
    if messages and messages[-1]["role"] == "assistant":
        content = ""
        for content_item in messages[-1]["content"]:
            if content_item.get("text"):
                content += content_item["text"]
        return content
    
    return "No response generated within max turns."

In [46]:
pokedex_loop("What is the combined weight of Ash's first 3 pokemons?")

--- Agent Loop Turn 1/4 ---
LLM requested tool calls:
LLM requested tool calls:
I'll help you find the combined weight of Ash's first 3 Pokémon. Ash's first three Pokémon were Pikachu, Caterpie, and Pidgeotto. Let me get their weight information.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'pikachu'}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60, 'height': 4, 'abilities': ['static', 'lightning-rod'], 'stats': {'hp': 35, 'attack': 55, 'defense': 40, 'special-attack': 50, 'special-defense': 50, 'speed': 90}}
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'caterpie'}
  - Response: {'name': 'Caterpie', 'id': 10, 'weight': 29, 'height': 3, 'abilities': ['shield-dust', 'run-away'], 'stats': {'hp': 45, 'attack': 30, 'defense': 35, 'special-attack': 20, 'special-defense': 20, 'speed': 45}}
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'pidgeotto'}
  - Response: {'name': 'Pidgeotto', 'id': 17, 'weight': 300, 'height': 11, 'abilities': ['keen-eye', 'tangled-feet', 'big

"The combined weight of Ash's first 3 Pokémon is **389 hectograms** (or 38.9 kg).\n\nHere's the breakdown:\n- Pikachu: 60 hectograms\n- Caterpie: 29 hectograms  \n- Pidgeotto: 300 hectograms\n- **Total: 389 hectograms**"

![](br_images/05_pokedex_loop.png)

This one is kind of an hallucination, I was expecting snorlax to be the heaviest pokemon.

In [47]:
pokedex_loop("Which is the heaviest pokemon?")

--- Agent Loop Turn 1/4 ---
LLM content response: I'd be happy to help you find the heaviest Pokémon! However, I don't have access to a tool that can search through all Pokémon to compare their weights. The tool I have available can only look up information for specific Pokémon by name or ID.

If you have some specific Pokémon in mind that you think might be among the heaviest, I can look up their weights and compare them for you. Some commonly known heavy Pokémon include:

- Groudon
- Dialga
- Giratina
- Wailord
- Snorlax

Would you like me to check the weights of any specific Pokémon, or do you have other candidates in mind that you'd like me to investigate?


"I'd be happy to help you find the heaviest Pokémon! However, I don't have access to a tool that can search through all Pokémon to compare their weights. The tool I have available can only look up information for specific Pokémon by name or ID.\n\nIf you have some specific Pokémon in mind that you think might be among the heaviest, I can look up their weights and compare them for you. Some commonly known heavy Pokémon include:\n\n- Groudon\n- Dialga\n- Giratina\n- Wailord\n- Snorlax\n\nWould you like me to check the weights of any specific Pokémon, or do you have other candidates in mind that you'd like me to investigate?"

## 5. Structuring the Agent with Classes

The loop above works, but for more complex agents, encapsulating the logic and state within classes is much better. We'll define:
- `AgentState`: A Pydantic model to hold the conversation history and potentially other state.
- `SimpleAgent`: A class containing the agent's configuration (model, system message, tools) and logic (`step`, `run`).

In [48]:
class AgentState(BaseModel):
    """Manages the state of the agent."""
    messages: List[Dict[str, Any]] = Field(default_factory=list)
    step_num: int = Field(default=0)
    final_assistant_content: str | None = None # Populated at the end of a run

class SimpleAgent:
    """A simple agent class with tracing, state, and tool processing."""
    def __init__(self, model_name: str, system_message: str, tools: List[Callable]):
        self.model_name = model_name
        self.system_message = system_message
        self.tools = [function_tool(t) for t in tools] # add schemas to the tools
    
    @weave.op(name="SimpleAgent.step") # Trace each step
    def step(self, state: AgentState) -> AgentState:
        step_num = state.step_num + 1
        messages = state.messages
        final_assistant_content = None
        
        try:
            # Create system and tool config for Bedrock
            system = [{"text": self.system_message}]
            tool_config = {"tools": [t.tool_schema for t in self.tools]}
            
            # call model with tools
            response = call_model(
                model_name=self.model_name, 
                messages=messages,
                system=system,
                toolConfig=tool_config
            )

            # add the assistant's response to the messages
            messages.append(response["output"]["message"])

            # if the LLM requested tool calls, perform them
            if response["stopReason"] == "tool_use":
                print("LLM requested tool calls:")
                # perform the tool calls
                tool_result_messages = perform_tool_calls(tools=self.tools, response=response)
                messages.append(tool_result_messages)
            
            # LLM gave content response
            elif response["stopReason"] == "end_turn":
                # Extract text content from Bedrock response
                content = ""
                for content_item in response["output"]["message"]["content"]:
                    if content_item.get("text"):
                        content += content_item["text"]
                final_assistant_content = content
            else:
                print(f"Unexpected stop reason: {response['stopReason']}")
                
        except Exception as e:
            print(f"ERROR in Agent Step: {e}")
            # Add an error message to history in Bedrock format
            error_message = {
                "role": "assistant", 
                "content": [{"text": f"Agent error in step: {str(e)}"}]
            }
            messages.append(error_message)
            final_assistant_content = f"Agent error in step {step_num}: {str(e)}"
            
        return AgentState(messages=messages, step_num=step_num, final_assistant_content=final_assistant_content)

    @weave.op(name="SimpleAgent.run")
    def run(self, user_prompt: str, max_turns: int = 10) -> AgentState:
        # Initialize state with Bedrock message format
        state = AgentState(messages=[
            {"role": "user", "content": [{"text": user_prompt}]}
        ])
        
        for _ in range(max_turns):
            print(f"--- Agent Loop Turn {state.step_num + 1}/{max_turns} ---")
            state = self.step(state)
            if state.final_assistant_content:
                return state
        return state

![](images/07_simple_agent.png)

In [49]:
agent = SimpleAgent(
    model_name=CLAUDE_MODEL,
    system_message="You are a helpful assistant that can use tools to answer questions.",
    tools=[get_pokemon_info, add_numbers]
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?")
print(f"Final response: {state.final_assistant_content}")


--- Agent Loop Turn 1/10 ---
LLM requested tool calls:
LLM requested tool calls:
I'll help you find the combined weight of Ash's first 3 Pokémon. Ash's first three Pokémon were Pikachu, Caterpie, and Pidgeotto. Let me get their weight information.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'pikachu'}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60, 'height': 4, 'abilities': ['static', 'lightning-rod'], 'stats': {'hp': 35, 'attack': 55, 'defense': 40, 'special-attack': 50, 'special-defense': 50, 'speed': 90}}
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'caterpie'}
  - Response: {'name': 'Caterpie', 'id': 10, 'weight': 29, 'height': 3, 'abilities': ['shield-dust', 'run-away'], 'stats': {'hp': 45, 'attack': 30, 'defense': 35, 'special-attack': 20, 'special-defense': 20, 'speed': 45}}
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'pidgeotto'}
  - Response: {'name': 'Pidgeotto', 'id': 17, 'weight': 300, 'height': 11, 'abilities': ['keen-eye', 'tangled-feet', 'bi

![](br_images/06_agent.png)

Possible improvements to the SimpleAgent:
- Give the model info about the state of the conversation, you could inject a message with the model context pressure, steps left, etc.
- Structured output. Make the model output a specific format, for instance a JSON with the expected fields.
- Add more tools like read and write files, access a database.
- Agent handoff: Agent1 does triage and Agent2 executes specific tasks.

# 6 Evaluating our Agent

We can evaluate our pokedex agent in multiple ways. One simple and powerful evaluation is analysing if the agent is making tool calls to retrieve the data or just replying from the LLM knowledge. This is common practice when building grounded knowledge bases, you want reponses to be from your documents and not just an LLM hallucination.

Let's implement this with [Weave Evaluations](https://weave-docs.wandb.ai/tutorial-eval)

In [50]:
eval_pokemons = [
    "Pikachu",
    "Bulbasaur",
    "Charmander",
    "Squirtle",
    "Jigglypuff",
    "Gengar",
    "Mewtwo",
    "Mew",
]

# let's retrieve some data using the api
pokemon_data = [get_pokemon_info(pokemon) for pokemon in eval_pokemons]
print(pokemon_data)

[{'name': 'Pikachu', 'id': 25, 'weight': 60, 'height': 4, 'abilities': ['static', 'lightning-rod'], 'stats': {'hp': 35, 'attack': 55, 'defense': 40, 'special-attack': 50, 'special-defense': 50, 'speed': 90}}, {'name': 'Bulbasaur', 'id': 1, 'weight': 69, 'height': 7, 'abilities': ['overgrow', 'chlorophyll'], 'stats': {'hp': 45, 'attack': 49, 'defense': 49, 'special-attack': 65, 'special-defense': 65, 'speed': 45}}, {'name': 'Charmander', 'id': 4, 'weight': 85, 'height': 6, 'abilities': ['blaze', 'solar-power'], 'stats': {'hp': 39, 'attack': 52, 'defense': 43, 'special-attack': 60, 'special-defense': 50, 'speed': 65}}, {'name': 'Squirtle', 'id': 7, 'weight': 90, 'height': 5, 'abilities': ['torrent', 'rain-dish'], 'stats': {'hp': 44, 'attack': 48, 'defense': 65, 'special-attack': 50, 'special-defense': 64, 'speed': 43}}, {'name': 'Jigglypuff', 'id': 39, 'weight': 55, 'height': 5, 'abilities': ['cute-charm', 'competitive', 'friend-guard'], 'stats': {'hp': 115, 'attack': 45, 'defense': 20, 

So this is ground truth data, we can use it to evaluate our agent. Create some questions that should be answered by the agent.

In [51]:
eval_dataset = [
  {"question": "What is the weight of Pikachu?", "answer": 60},
  {"question": "What is Pikachu's height?", "answer": 4},
  {"question": "How many abilities does Bulbasaur have?", "answer": 2},
  {"question": "List Bulbasaur's abilities.", "answer": ["overgrow", "chlorophyll"]},
  {"question": "What is the speed stat of Charmander?",   "answer": 65},
  {"question": "What is Jigglypuff's HP?","answer": 115},
  {"question": "What is the defense stat of Squirtle?", "answer": 65},
  {"question": "Is Gengar's attack greater than 70?", "answer": "No"},
  {"question": "What are Mewtwo's abilities?", "answer": ["pressure", "unnerve"]},
  {"question": "What is the id of Mew?", "answer": 151},
  {"question": "Is Squirtle a water type?", "answer": "There is no info about this on the database"},
  {"question": "Which region does Gengar come from?", "answer": "There is no info about this on the database"},
  {"question": "What color is Pikachu?", "answer": "There is no info about this on the database"},
  {"question": "At what level does Charmander evolve?", "answer": "There is no info about this on the database"},
  {"question": "What item does Mew hold by default?", "answer": "There is no info about this on the database"},
  {"question": "What is the height of Mew?", "answer": 4},
  {"question": "Does Bulbasaur have the ability 'solar-power'?", "answer": "No"},
  {"question": "What is the special-defense of Mewtwo?", "answer": 90},
  {"question": "Is Jigglypuff's speed higher than Pikachu's?", "answer": "No"},
  {"question": "How much does Gengar weigh?", "answer": 405}
]

In [52]:
pokedex_agent = SimpleAgent(
    model_name=CLAUDE_MODEL,
    system_message="You are a helpful assistant that can use tools to answer questions.",
    tools=[get_pokemon_info, add_numbers]
)

In [53]:
state = pokedex_agent.run(user_prompt=eval_dataset[0]["question"])
pprint(state)

--- Agent Loop Turn 1/10 ---
LLM requested tool calls:
LLM requested tool calls:
I'll help you find the weight of Pikachu by looking up its information.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'Pikachu'}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60, 'height': 4, 'abilities': ['static', 'lightning-rod'], 'stats': {'hp': 35, 'attack': 55, 'defense': 40, 'special-attack': 50, 'special-defense': 50, 'speed': 90}}
--- Agent Loop Turn 2/10 ---


AgentState(
│   messages=[
│   │   {'role': 'user', 'content': [{'text': 'What is the weight of Pikachu?'}]},
│   │   {
│   │   │   'role': 'assistant',
│   │   │   'content': [
│   │   │   │   {'text': "I'll help you find the weight of Pikachu by looking up its information."},
│   │   │   │   {
│   │   │   │   │   'toolUse': {
│   │   │   │   │   │   'toolUseId': 'tooluse_6v75smm-S0CyKTfkAweHYw',
│   │   │   │   │   │   'name': 'get_pokemon_info',
│   │   │   │   │   │   'input': {'pokemon_name': 'Pikachu'}
│   │   │   │   │   }
│   │   │   │   }
│   │   │   ]
│   │   },
│   │   {
│   │   │   'role': 'user',
│   │   │   'content': [
│   │   │   │   {
│   │   │   │   │   'toolResult': {
│   │   │   │   │   │   'toolUseId': 'tooluse_6v75smm-S0CyKTfkAweHYw',
│   │   │   │   │   │   'status': 'success',
│   │   │   │   │   │   'content': [
│   │   │   │   │   │   │   {
│   │   │   │   │   │   │   │   'json': {
│   │   │   │   │   │   │   │   │   'result': {
│   │   │   │   │   │   │   │   │   │   'name': 'Pikachu',
│   │   │   │   │   │   │   │   │   │   'id': 25,
│   │   │   │   │   │   │   │   │   │   'weight': 60,
│   │   │   │   │   │   │   │   │   │   'height': 4,
│   │   │   │   │   │   │   │   │   │   'abilities': ['static', 'lightning-rod'],
│   │   │   │   │   │   │   │   │   │   'stats': {
│   │   │   │   │   │   │   │   │   │   │   'hp': 35,
│   │   │   │   │   │   │   │   │   │   │   'attack': 55,
│   │   │   │   │   │   │   │   │   │   │   'defense': 40,
│   │   │   │   │   │   │   │   │   │   │   'special-attack': 50,
│   │   │   │   │   │   │   │   │   │   │   'special-defense': 50,
│   │   │   │   │   │   │   │   │   │   │   'speed': 90
│   │   │   │   │   │   │   │   │   │   }
│   │   │   │   │   │   │   │   │   }
│   │   │   │   │   │   │   │   }
│   │   │   │   │   │   │   }
│   │   │   │   │   │   ]
│   │   │   │   │   }
│   │   │   │   }
│   │   │   ]
│   │   },
│   │   {
│   │   │   'role': 'assistant',
│   │   │   'content': [
│   │   │   │   {'text': 'Pikachu weighs 60 hectograms, which is equivalent to 6 kilograms (about 13.2 pounds).'}
│   │   │   ]
│   │   }
│   ],
│   step_num=2,
│   final_assistant_content='Pikachu weighs 60 hectograms, which is equivalent to 6 kilograms (about 13.2 pounds).'
)

Maybe we need to remove this hectogram thing...

In [54]:
# subclass of pydantic model
class PokedexAgent(weave.Model):
    system_message: str
    tools: list[Callable]
    model_name: str
    # you could have other model parameters here, like temperature, top_p, etc.
    # they would be versioned and stored in the model state

    @weave.op
    def predict(self, question: str) -> str:
        agent = SimpleAgent(
            model_name=self.model_name,
            system_message=self.system_message,
            tools=self.tools
        )
        state = agent.run(user_prompt=question) 
        return {"response": state.final_assistant_content, "step_num": state.step_num }

In [55]:
pokedex_agent = PokedexAgent(
    model_name=CLAUDE_MODEL,
    system_message="You are a helpful assistant that can use tools to answer questions.",
    tools=[get_pokemon_info, add_numbers]
)

pokedex_agent.predict(question="What is the weight of Pikachu?")

--- Agent Loop Turn 1/10 ---
LLM requested tool calls:
LLM requested tool calls:
I'll help you find the weight of Pikachu by looking up its information.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'Pikachu'}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60, 'height': 4, 'abilities': ['static', 'lightning-rod'], 'stats': {'hp': 35, 'attack': 55, 'defense': 40, 'special-attack': 50, 'special-defense': 50, 'speed': 90}}
--- Agent Loop Turn 2/10 ---


{'response': 'Pikachu weighs 60 hectograms. Since 1 hectogram equals 100 grams, this means Pikachu weighs 6 kilograms (or about 13.2 pounds).',
 'step_num': 2}

In [56]:
@weave.op
def accuracy(output, answer) -> bool:
    response = output["response"]
    if isinstance(answer, str):
        return answer in response
    elif isinstance(answer, list):
        return set(response) == set(answer)
    else:
        return str(answer) in str(response)

In [57]:
evaluation = weave.Evaluation(dataset=eval_dataset, scorers=[accuracy])

Let's run it

In [58]:
results = await evaluation.evaluate(pokedex_agent)

--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
LLM requested tool calls:
LLM requested tool calls:
I'll look up Pikachu's weight for you.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'Pikachu'}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60, 'height': 4, 'abilities': ['static', 'lightning-rod'], 'stats': {'hp': 35, 'attack': 55, 'defense': 40, 'special-attack': 50, 'special-defense': 50, 'speed': 90}}
--- Agent Loop Turn 2/10 ---
LLM requested too

Evaluated 1 of 20 examples

LLM requested tool calls:
LLM requested tool calls:
I'll look up information about Mew for you.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'Mew'}
LLM requested tool calls:
LLM requested tool calls:
I'll look up Mewtwo's information to find its special defense stat.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'Mewtwo'}
LLM requested tool calls:
LLM requested tool calls:
I'll look up Pikachu's information for you.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'Pikachu'}
LLM requested tool calls:
LLM requested tool calls:
I'll look up Squirtle's information to find its defense stat.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'Squirtle'}
LLM requested tool calls:
LLM requested tool calls:
I'll look up Bulbasaur's information to check if it has the 'solar-power' ability.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'Bulbasaur'}
LLM requested tool calls:
LLM requested tool calls:
I'll look up information about Squirtle to check its type for you.
  - Tool: ge

Evaluated 2 of 20 examples

Evaluated 3 of 20 examples

...

Evaluated 17 of 20 examples

Evaluated 18 of 20 examples

Evaluated 19 of 20 examples

Evaluated 20 of 20 examples

Evaluation summary
{
    'output': {'step_num': {'mean': 2.0}},
    'accuracy': {'true_count': 12, 'true_fraction': 0.6},
    'model_latency': {'mean': 7.462940955162049}
}

Better agent prompting

In [59]:
pokedex_agent2 = PokedexAgent(
    model_name=CLAUDE_MODEL,
    system_message="You are a helpful assistant that can use tools to answer questions. If the questions is not possible to answer, you should say 'There is no info about this on the database', nothing else.",
    tools=[get_pokemon_info, add_numbers]
)

In [60]:
results = await evaluation.evaluate(pokedex_agent2)

--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
--- Agent Loop Turn 1/10 ---
LLM requested tool calls:
LLM requested tool calls:
I'll look up Mewtwo's abilities for you.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'Mewtwo'}
  - Response: {'name': 'Mewtwo', 'id': 150, 'weight': 1220, 'height': 20, 'abilities': ['pressure', 'unnerve'], 'stats': {'hp': 106, 'attack': 110, 'defense': 90, 'special-attack': 154, 'special-defense': 90, 'speed': 130}}
--- Agent Loop Turn 2/10 ---
LLM requested

Evaluated 1 of 20 examples

Evaluated 2 of 20 examples

Evaluated 3 of 20 examples

Evaluated 4 of 20 examples

...

Evaluated 17 of 20 examples

Evaluated 18 of 20 examples

Evaluated 19 of 20 examples

Evaluated 20 of 20 examples

Evaluation summary
{
    'output': {'step_num': {'mean': 2.0}},
    'accuracy': {'true_count': 17, 'true_fraction': 0.85},
    'model_latency': {'mean': 5.016485893726349}
}

![](br_images/07_eval.png)
Ok this is considerably better!